In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

In [0]:
# para gerar o grafico no browser
%matplotlib inline

In [0]:
# local dos dados e modifica o diretorio
!git clone https://github.com/Jeniffer2018/Redes-Neurais
% cd Redes-Neurais
! ls

Cloning into 'Redes-Neurais'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 57 (delta 1), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (57/57), done.


In [0]:
# leitura dos dados
DAT=pd.read_csv("dados.csv")
print(DAT.head())

In [0]:
# separa as colunas de interesse
Index=[0,1,2,3,4,5]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()
print(Dat_Select.head())

In [0]:
# funcao normatiza dados intervalo de -1 a 1
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)
print(DAT_OLD.head())
print(DAT.head())

In [0]:
# separa os dados em treino e teste
train=DAT.sample(frac=0.5,random_state=2) # o artigo usou metade dos dados para o treino
test=DAT.drop(train.index)

In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1,2,3,4]]
x_test=test.iloc[:,[0,1,2,3,4]]
X_OLD=DAT_OLD.iloc[:,[0,1,2,3,4]]
print(x_train.head())
print(x_test.head())
print(X_OLD.head())

In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[5]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
print(y_train.head())
print(y_test.head())
print(Y_OLD.head())

In [0]:
# define a rede neural 
# o artigo usou 9 neurônios na camada oculta 
clf = MLPRegressor(solver='lbfgs',activation='logistic',alpha=1e-5,hidden_layer_sizes=(9), 
                    random_state=None)

In [0]:
# treina a rede neural
clf.fit(x_train, y_train)

In [0]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(x_train)
y_calc_test=clf.predict(x_test)

In [0]:
# transforma a saida da rede em dataframe 
y_calc_train=pd.DataFrame(y_calc_train)
y_calc_test=pd.DataFrame(y_calc_test)
col_names=list(y_train)
y_calc_train.columns = col_names
y_calc_test.columns = col_names

In [0]:
# transforma dos dados calculados na forma original (nao normatizado)
y_calc_train=Original(y_calc_train,Y_OLD)
y_calc_test=Original(y_calc_test,Y_OLD)

In [0]:
# recupera os dados originais 
test=[]
train=[]
train=Original(y_train,Y_OLD)
test=Original(y_test,Y_OLD)

In [0]:
# escolhe uma das resposta para fazer o grafico (resistencia-treino)
resist_obs_train=[]
resist_calc_train=[]
resist_calc_train=y_calc_train['f3'].copy()
resist_obs_train=train['f3'].copy()
print(resist_calc_train.head())
print(resist_obs_train.head())

In [0]:
# escolhe uma das resposta para fazer o grafico (resistencia-teste)
resist_obs_test=[]
resist_calc_test=[]
resist_calc_test=y_calc_test['f3'].copy()
resist_obs_test=test['f3'].copy()
print(resist_calc_test.head())
print(resist_obs_test.head())

In [0]:
# expressa os dois dados em figura
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(resist_obs_test,resist_calc_test, s=10, c='b', marker="s", label='teste')
ax1.scatter(resist_obs_train,resist_calc_train, s=10, c='r', marker="o", label='treino')
plt.legend(loc='upper left')
plt.ylabel('f3 Calc')
plt.xlabel('f3 Obs')
plt.show()

In [0]:
# cálculo de MSE e R² para o teste
mse=mean_squared_error(resist_obs_test,resist_calc_test)
print("MSE teste=",mse)
R2=r2_score(resist_obs_test,resist_calc_test)
print("R^2 teste=",R2)

In [0]:
# cálculo de MSE e R² para o treino
mse=mean_squared_error(resist_obs_train,resist_calc_train)
print("MSE treino=",mse)
R2=r2_score(resist_obs_train,resist_calc_train)
print("R^2 treino=",R2)